In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import random
import tqdm
import pyautogui
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By
import time 
import datetime

In [2]:
browser = webdriver.Chrome()
df = pd.DataFrame(columns = ['Что ищем','Время сбора','Ссылка','adr','work','ost','otz','osob','inf'])

In [3]:
browser.get(r'https://yandex.ru/maps/')

In [4]:
Poisk = browser.find_element(By.CLASS_NAME,'input__control')
adress = pd.read_excel('адр_пробник.xlsx')
adress = list(adress['адреса'])
adress

['г. казань, ул. Петербургская, 28, отделение сбербанка',
 'ул. Пушкина, 4, Казань, отделение сбербанка']

In [ ]:
def sbor(aa):
    temp = pd.DataFrame(columns = ['Что ищем','Время сбора','Ссылка','adr','work','ost','otz','osob','inf'])
    time.sleep(1)
    
    work = ''
    
    if browser.find_elements(By.CLASS_NAME,'card-feature-view__value')[0].text == 'Добавить':
        work = browser.find_elements(By.CLASS_NAME,'business-working-status-view')[-1].text
    elif browser.find_elements(By.CLASS_NAME,'card-feature-view__value')[0].text == 'График':
        browser.find_elements(By.CLASS_NAME,'card-feature-view__value')[0].click()
        time.sleep(1)
        #browser.find_elements(By.CLASS_NAME,'business-working-status-view')[0].click()
        time.sleep(2)
        e = browser.find_elements(By.CLASS_NAME,'business-working-intervals-view__item')
        
        for ee in e:
            work = work + ' ' + ee.text
        time.sleep(1)

    adr = browser.find_elements(By.CLASS_NAME,'business-contacts-view__address-link')
    adr = adr[0].text

    ost = browser.find_element(By.CLASS_NAME,'masstransit-stops-view__stop-name')
    metrs = browser.find_element(By.CLASS_NAME,'masstransit-stops-view__stop-distance')
    ost = ost.text+'  '+metrs.text

    features = browser.find_elements(By.CLASS_NAME,'business-features-view__bool-text')

    vkl = browser.find_elements(By.CLASS_NAME,'tabs-select-view__title')

    vkl[2].click()
    time.sleep(5)

    o = browser.find_elements(By.CLASS_NAME,'business-review-view__body-text')
    otz = []
    for i in range(len(o)):
        otz.append(o[i].text)

    vkl[4].click()
    time.sleep(5)

    o = browser.find_elements(By.CLASS_NAME,'business-features-view__bool-text')
    osob = []
    for i in range(len(o)):
        osob.append(o[i].text)

    time.sleep(3)

    o = browser.find_elements(By.CLASS_NAME,'business-features-view__valued-value')
    oper = []
    for i in range(len(o)):
        oper.append(o[i].text)
    temp = temp.append({'Что ищем':aa,'Время сбора':datetime.datetime.now(),'Ссылка':browser.current_url,'adr':adr,'work':work,'ost':ost,'otz':otz,'osob':osob,'inf':oper},ignore_index=True)
    return temp

for aa in adress:
    Poisk.send_keys(Keys.CONTROL + 'a')
    Poisk.send_keys(Keys.DELETE)
    time.sleep(0.5)
    
    
    Poisk.send_keys(aa)
    time.sleep(1.5)
    Poisk.send_keys(Keys.ENTER)
    time.sleep(5)
    h = ''
    elem = browser.find_elements(By.CLASS_NAME,'search-business-snippet-view')
    if len(elem)>0:
        while h != elem[-1]:
            h = elem[-1]
            browser.execute_script("arguments[0].scrollIntoView(true);",elem[-1])
            time.sleep(1)
            elem = browser.find_elements(By.CLASS_NAME,'search-business-snippet-view')
        
    
        for h in elem:
            browser.execute_script("arguments[0].scrollIntoView(true);",h)
            time.sleep(1)
            h.click()
            df = df.append(sbor(aa))
            
    else:
        df = df.append(sbor(aa))
            

df